In [1]:
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit

from qiskit.primitives import BackendEstimator,Estimator,Sampler
from qiskit import Aer

from squlearn.util.optree import *
from squlearn.util.optree.optree import *
from squlearn.util.optree.optree_derivative import *
from squlearn.util.optree.optree_evaluate import *

#circuit = random_circuit(4, 2, seed=12)
#circuit.draw()

pqc_param = ParameterVector('θ', 4)

circuit = QuantumCircuit(4)
circuit.h([0,1,2,3])
circuit.u(0.5,0.5,0.5,[0,1,2,3])
circuit.cx(0,1)
circuit.cx(2,3)
circuit.cx(1,2)

circuit2 = circuit.copy()

circuit.ry(pqc_param[0],0)
circuit.ry(pqc_param[1],1)
circuit.ry(pqc_param[2],2)
circuit.ry(pqc_param[3],3)
circuit.draw()

circuit2.rx(pqc_param[0],0)
circuit2.rx(pqc_param[1],1)
circuit2.rx(pqc_param[2],2)
circuit2.rx(pqc_param[3],3)



In [2]:
param1 = {pqc_param[0]:0.5,pqc_param[1]:0.5,pqc_param[2]:0.5,pqc_param[3]:0.5}
param2 = {pqc_param[0]:0.15,pqc_param[1]:0.15,pqc_param[2]:0.85,pqc_param[3]:0.95}


In [3]:
op_param = ParameterVector('p',3)
op = SparsePauliOp(["XXXX","YYYY","IIYY"],[op_param[0],op_param[1],op_param[2]])
op_dict1={op_param[0]:0.5,op_param[1]:0.5,op_param[2]:0.5}
op_dict2={op_param[0]:0.25,op_param[1]:0.25,op_param[2]:0.25}

In [4]:
op.num_qubits
op_clean = SparsePauliOp(["ZZZZ","ZZZI","IIZZ"],[op_param[0],op_param[1],op_param[2]])
op_clean_param=ParameterVector('p',3)
op_clean_dict1={op_clean_param[0]:0.5,op_clean_param[1]:0.2,op_clean_param[2]:0.3}
op_clean_dict2={op_clean_param[0]:0.12,op_clean_param[1]:0.8,op_clean_param[2]:-0.3}

In [5]:
opdic1 = copy.copy(op_dict1)
opdic1.update(op_clean_dict1)

opdic2 = copy.copy(op_dict2)
opdic2.update(op_clean_dict2)


In [6]:
test = transform_operator_to_zbasis(op)
print(test)

(1.0*
     ┌───┐┌─┐         
q_0: ┤ H ├┤M├─────────
     ├───┤└╥┘┌─┐      
q_1: ┤ H ├─╫─┤M├──────
     ├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├─╫──╫─┤M├───
     ├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├─╫──╫──╫─┤M├
     └───┘ ║  ║  ║ └╥┘
c: 4/══════╩══╩══╩══╩═
           0  1  2  3 

 with observable 
SparsePauliOp(['ZZZZ'],
              coeffs=[ParameterExpression(1.0*p[0])])
 + 1.0*
     ┌─────┐┌───┐┌─┐         
q_0: ┤ Sdg ├┤ H ├┤M├─────────
     ├─────┤├───┤└╥┘┌─┐      
q_1: ┤ Sdg ├┤ H ├─╫─┤M├──────
     ├─────┤├───┤ ║ └╥┘┌─┐   
q_2: ┤ Sdg ├┤ H ├─╫──╫─┤M├───
     ├─────┤├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ Sdg ├┤ H ├─╫──╫──╫─┤M├
     └─────┘└───┘ ║  ║  ║ └╥┘
c: 4/═════════════╩══╩══╩══╩═
                  0  1  2  3 

 with observable 
SparsePauliOp(['ZZZZ', 'IIZZ'],
              coeffs=[ParameterExpression(1.0*p[1]), ParameterExpression(1.0*p[2])])
)


In [7]:
print(gen_expectation_tree(circuit,test))

(1.0*
     ┌───┐┌────────────────┐     ┌──────────┐   ┌───┐    ┌─┐           
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ Ry(θ[0]) ├───┤ H ├────┤M├───────────
     ├───┤├────────────────┤┌─┴─┐└──────────┘┌──┴───┴───┐└╥┘┌───┐┌─┐   
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─────■──────┤ Ry(θ[1]) ├─╫─┤ H ├┤M├───
     ├───┤├────────────────┤└───┘   ┌─┴─┐    ├──────────┤ ║ ├───┤└╥┘┌─┐
q_2: ┤ H ├┤ U(0.5,0.5,0.5) ├──■─────┤ X ├────┤ Ry(θ[2]) ├─╫─┤ H ├─╫─┤M├
     ├───┤├────────────────┤┌─┴─┐┌──┴───┴───┐└──┬───┬───┘ ║ └┬─┬┘ ║ └╥┘
q_3: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├┤ Ry(θ[3]) ├───┤ H ├─────╫──┤M├──╫──╫─
     └───┘└────────────────┘└───┘└──────────┘   └───┘     ║  └╥┘  ║  ║ 
c: 4/═════════════════════════════════════════════════════╩═══╩═══╩══╩═
                                                          0   3   1  2 

 with observable 
SparsePauliOp(['ZZZZ'],
              coeffs=[ParameterExpression(1.0*p[0])])
 + 1.0*
     ┌───┐┌────────────────┐     ┌──────────┐  ┌─────┐    ┌───┐ ┌─┐           
q_0: ┤ H ├┤ U(0.5,

In [8]:
tree = OpTreeLeafExpectationValue(circuit,op)
print(tree)


     ┌───┐┌────────────────┐     ┌──────────┐            
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ Ry(θ[0]) ├────────────
     ├───┤├────────────────┤┌─┴─┐└──────────┘┌──────────┐
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─────■──────┤ Ry(θ[1]) ├
     ├───┤├────────────────┤└───┘   ┌─┴─┐    ├──────────┤
q_2: ┤ H ├┤ U(0.5,0.5,0.5) ├──■─────┤ X ├────┤ Ry(θ[2]) ├
     ├───┤├────────────────┤┌─┴─┐┌──┴───┴───┐└──────────┘
q_3: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├┤ Ry(θ[3]) ├────────────
     └───┘└────────────────┘└───┘└──────────┘            

 with observable 
SparsePauliOp(['XXXX', 'YYYY', 'IIYY'],
              coeffs=[ParameterExpression(1.0*p[0]), ParameterExpression(1.0*p[1]),
 ParameterExpression(1.0*p[2])])



In [9]:
adjust_tree = transform_tree_to_zbasis(tree,abelian_grouping=False)
print(adjust_tree)

(1.0*
     ┌───┐┌────────────────┐     ┌──────────┐   ┌───┐    ┌─┐           
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ Ry(θ[0]) ├───┤ H ├────┤M├───────────
     ├───┤├────────────────┤┌─┴─┐└──────────┘┌──┴───┴───┐└╥┘┌───┐┌─┐   
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─────■──────┤ Ry(θ[1]) ├─╫─┤ H ├┤M├───
     ├───┤├────────────────┤└───┘   ┌─┴─┐    ├──────────┤ ║ ├───┤└╥┘┌─┐
q_2: ┤ H ├┤ U(0.5,0.5,0.5) ├──■─────┤ X ├────┤ Ry(θ[2]) ├─╫─┤ H ├─╫─┤M├
     ├───┤├────────────────┤┌─┴─┐┌──┴───┴───┐└──┬───┬───┘ ║ └┬─┬┘ ║ └╥┘
q_3: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├┤ Ry(θ[3]) ├───┤ H ├─────╫──┤M├──╫──╫─
     └───┘└────────────────┘└───┘└──────────┘   └───┘     ║  └╥┘  ║  ║ 
c: 4/═════════════════════════════════════════════════════╩═══╩═══╩══╩═
                                                          0   3   1  2 

 with observable 
SparsePauliOp(['ZZZZ'],
              coeffs=[ParameterExpression(1.0*p[0])])
 + 1.0*
     ┌───┐┌────────────────┐     ┌──────────┐  ┌─────┐    ┌───┐ ┌─┐           
q_0: ┤ H ├┤ U(0.5,

In [10]:
evaluate_estimator(OpTreeNodeList([circuit,circuit2]),OpTreeNodeList([op_clean,op,op_clean]),[param1,param2],[opdic1,opdic2],Estimator(),dictionaries_combined=False)

pre-processing 0.0030138492584228516
run time 0.03871726989746094
post processing 0.0


array([[[[-0.06118615, -0.26725525, -0.06118615],
         [ 0.0742432 , -0.25791181,  0.0742432 ]],

        [[-0.03059308, -0.13362763, -0.03059308],
         [ 0.0371216 , -0.1289559 ,  0.0371216 ]]],


       [[[-0.22534089, -0.17570876, -0.22534089],
         [-0.00219049, -0.06085567, -0.00219049]],

        [[-0.11267045, -0.08785438, -0.11267045],
         [-0.00109525, -0.03042783, -0.00109525]]]])

In [11]:
evaluate_sampler_v2(OpTreeNodeList([circuit,circuit2]),OpTreeNodeList([op_clean,test,op_clean]),param1,opdic1,Sampler(),dictionaries_combined=False)

measure_list [None, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001D12211F1C0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001D12211E620>, None]
measure_index_list [[0, 3], [1], [2]]
input: [1.0*
     ┌───┐┌────────────────┐     ┌──────────┐            
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ Ry(θ[0]) ├────────────
     ├───┤├────────────────┤┌─┴─┐└──────────┘┌──────────┐
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─────■──────┤ Ry(θ[1]) ├
     ├───┤├────────────────┤└───┘   ┌─┴─┐    ├──────────┤
q_2: ┤ H ├┤ U(0.5,0.5,0.5) ├──■─────┤ X ├────┤ Ry(θ[2]) ├
     ├───┤├────────────────┤┌─┴─┐┌──┴───┴───┐└──────────┘
q_3: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├┤ Ry(θ[3]) ├────────────
     └───┘└────────────────┘└───┘└──────────┘            
, 1.0*
     ┌───┐┌────────────────┐     ┌──────────┐            
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ Rx(θ[0]) ├────────────
     ├───┤├────────────────┤┌─┴─┐└──────────┘┌──────────┐
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─────■──────┤ Rx(θ[1]) ├
     ├──

array([[-0.06118615, -0.26725525, -0.06118615],
       [ 0.0742432 , -0.25791181,  0.0742432 ]])

In [12]:
evaluate_sampler_v2(OpTreeNodeList([circuit,circuit2]),OpTreeNodeList([op_clean,test,op_clean]),param2,opdic2,Sampler(),dictionaries_combined=False)

measure_list [None, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001D12211F1C0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001D12211E620>, None]
measure_index_list [[0, 3], [1], [2]]
input: [1.0*
     ┌───┐┌────────────────┐     ┌──────────┐            
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ Ry(θ[0]) ├────────────
     ├───┤├────────────────┤┌─┴─┐└──────────┘┌──────────┐
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─────■──────┤ Ry(θ[1]) ├
     ├───┤├────────────────┤└───┘   ┌─┴─┐    ├──────────┤
q_2: ┤ H ├┤ U(0.5,0.5,0.5) ├──■─────┤ X ├────┤ Ry(θ[2]) ├
     ├───┤├────────────────┤┌─┴─┐┌──┴───┴───┐└──────────┘
q_3: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├┤ Ry(θ[3]) ├────────────
     └───┘└────────────────┘└───┘└──────────┘            
, 1.0*
     ┌───┐┌────────────────┐     ┌──────────┐            
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ Rx(θ[0]) ├────────────
     ├───┤├────────────────┤┌─┴─┐└──────────┘┌──────────┐
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─────■──────┤ Rx(θ[1]) ├
     ├──

array([[-0.11267045, -0.08785438, -0.11267045],
       [-0.00109525, -0.03042783, -0.00109525]])

In [13]:
evaluate_sampler_v2(OpTreeNodeList([circuit,circuit2]),OpTreeNodeList([op_clean,test,op_clean]),[param1,param2],[opdic1,opdic2],Sampler(),dictionaries_combined=False)

measure_list [None, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001D12211F1C0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001D12211E620>, None]
measure_index_list [[0, 3], [1], [2]]
input: [1.0*
     ┌───┐┌────────────────┐     ┌──────────┐            
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ Ry(θ[0]) ├────────────
     ├───┤├────────────────┤┌─┴─┐└──────────┘┌──────────┐
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─────■──────┤ Ry(θ[1]) ├
     ├───┤├────────────────┤└───┘   ┌─┴─┐    ├──────────┤
q_2: ┤ H ├┤ U(0.5,0.5,0.5) ├──■─────┤ X ├────┤ Ry(θ[2]) ├
     ├───┤├────────────────┤┌─┴─┐┌──┴───┴───┐└──────────┘
q_3: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├┤ Ry(θ[3]) ├────────────
     └───┘└────────────────┘└───┘└──────────┘            
, 1.0*
     ┌───┐┌────────────────┐     ┌──────────┐            
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ Rx(θ[0]) ├────────────
     ├───┤├────────────────┤┌─┴─┐└──────────┘┌──────────┐
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─────■──────┤ Rx(θ[1]) ├
     ├──

array([[[[-0.06118615, -0.26725525, -0.06118615],
         [ 0.0742432 , -0.25791181,  0.0742432 ]],

        [[-0.03059308, -0.13362763, -0.03059308],
         [ 0.0371216 , -0.1289559 ,  0.0371216 ]]],


       [[[-0.22534089, -0.17570876, -0.22534089],
         [-0.00219049, -0.06085567, -0.00219049]],

        [[-0.11267045, -0.08785438, -0.11267045],
         [-0.00109525, -0.03042783, -0.00109525]]]])

In [14]:
evaluate_sampler_v2(OpTreeNodeList([circuit,circuit2]),OpTreeNodeList([op_clean,test,op_clean]),[param1,param2],[opdic1,opdic2],Sampler(),dictionaries_combined=True)

measure_list [None, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001D12211F1C0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001D12211E620>, None]
measure_index_list [[0, 3], [1], [2]]
input: [1.0*
     ┌───┐┌────────────────┐     ┌──────────┐            
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ Ry(θ[0]) ├────────────
     ├───┤├────────────────┤┌─┴─┐└──────────┘┌──────────┐
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─────■──────┤ Ry(θ[1]) ├
     ├───┤├────────────────┤└───┘   ┌─┴─┐    ├──────────┤
q_2: ┤ H ├┤ U(0.5,0.5,0.5) ├──■─────┤ X ├────┤ Ry(θ[2]) ├
     ├───┤├────────────────┤┌─┴─┐┌──┴───┴───┐└──────────┘
q_3: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├┤ Ry(θ[3]) ├────────────
     └───┘└────────────────┘└───┘└──────────┘            
, 1.0*
     ┌───┐┌────────────────┐     ┌──────────┐            
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ Rx(θ[0]) ├────────────
     ├───┤├────────────────┤┌─┴─┐└──────────┘┌──────────┐
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─────■──────┤ Rx(θ[1]) ├
     ├──

array([[[-0.06118615, -0.26725525, -0.06118615],
        [ 0.0742432 , -0.25791181,  0.0742432 ]],

       [[-0.11267045, -0.08785438, -0.11267045],
        [-0.00109525, -0.03042783, -0.00109525]]])

In [15]:
#evaluate_expectation_tree_from_sampler(tree,{},Sampler())

In [16]:
evaluate_expectation_tree_from_sampler(adjust_tree,{},Sampler())

KeyError: ParameterVectorElement(θ[0])

In [ ]:
job = Estimator().run(circuit,op)
job.result().values

In [ ]:
evaluate_expectation_tree_from_estimator(tree,{},Estimator())

In [ ]:
# Create a backend estimator
backend = Aer.get_backend('statevector_simulator')
estimator = BackendEstimator(backend=backend,options={'shots':1000000})

job = estimator.run(circuit,op)

In [ ]:
job.result().values